In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import urban_dictionary_scraper
import torch
import re
import sys
import pickle
import wiki_article 
import dictionary_definition
import glob
import modeling
import itertools
import random
from dataclasses import dataclass
from io import StringIO
from transformers import AutoModelWithLMHead, AutoTokenizer
from scipy import stats
import hashlib
from collections import OrderedDict

In [ ]:
def get_checkpoints(base_dir):
    checkpoint_dirs = glob.glob(f"{base_dir}/checkpoint*")
    checkpoint_dirs.sort(key=lambda x: int(x[(x.index("checkpoint-") + len("checkpoint-")):]))
    return checkpoint_dirs
modeling_gpt
def evaluate_lm_checkpoints(base_dir, validation_path):
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    for d in get_checkpoints(base_dir):
        model = AutoModelWithLMHead.from_pretrained(d).to('cuda')
        refined_model_eval = wiki_article.lm_eval(model, tokenizer, validation_path)
        print(f"{d}: {refined_model_eval}")
tokenizer
def evaluate_title_checkpoints(base_dir, validation_path):
    tokenizer = AutoTokenizer.from_pretrained("gpt2")print(parsed_page.body.prettify())
    for d in get_checkpoints(base_dir):
        model = AutoModelWithLMHead.from_pretrained(d).to('cuda')
        refined_model_eval = wiki_article.run_title_evaluation(model, tokenizer, validation_path)
        print(f"{d}: m={refined_model_eval.mean}, v={refined_model_eval.variance}")

# evaluate_lm_checkAutoModelWithLMHead, AutoTokenizer, points("models/wikitext_103_stride_512_v0/", "data/wikitext-103-title-train/wiki_title.valid.raw")
#print(glob.glob("models/wikitext_103_stride_512_v0/*"))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens(urban_dictionary_scraper.SpecialTokens.special_tokens_dict())

In [ ]:
with open("data/all_words.pickle", "rb") as f:
    #words = pickle.load(f)
    #items = list(words.items())
    random.shuffle(items)
    items = OrderedDict(items)
    
with open("data/all_words_randomized.pickle", "wb") as f:
    pickle.dump(items, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
urban_dictionary_scraper.UrbanDictionaryDataset._make_examples(tokenizer, words[2])

In [ ]:

model = AutoModelWithLMHead.from_pretrained("gpt2").to('cuda')

In [ ]:
unrefined_model_eval = wiki_article.run_title_evaluation(model, tokenizer, "wikitext-103-raw/wiki.valid.raw")
unrefined_model_eval

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelWithLMHead.from_pretrained("output_103/").to('cuda')

In [ ]:
refined_model_eval = wiki_article.run_title_evaluation(model, tokenizer, "wikitext-103-raw/wiki.valid.raw")
refined_model_eval

In [ ]:
sequence = f"\"TITLE\" is a song collaboration by Chinese artist Pamela Chen and Canadian singer Thomas Dimson, first released independently in March 2020. After gaining popularity amongst the cat community, the single was re-released by major label Columbia Records in May 2020. Pamela describes the song as being originally inspired by her two kittens, Apollo and Bean who once said meow.<bot>"

model =  modeling.GPT2LMHeadWithWeightedLossModel.from_pretrained("models/wikitext-103-raw-title-scale-20-lr5e-5").to("cuda")
input = tokenizer.encode(sequence, return_tensors="pt").to('cuda')
generated = model.generate(input, max_length=100, num_return_sequences=100, temperature=1)

print(f"Prompt text: {sequence}")
for i in range(generated.size()[0]):
    sentence_tokens = generated[i, :].tolist()
    decoded = tokenizer.decode(sentence_tokens)
    m = re.search(r"<bot>(.*?)<eot>", decoded)
    if m:
        print(f"{i}) {m.groups(1)}")
    else:
        print(f"{i}) Didn't work")
    

resulting_string = tokenizer.decode(generated.tolist()[0])
# print(resulting_string)

In [ ]:
for entry in entries: 
    m = re.match(r"\s*" + re.escape(entry.title) + r"\d*\s*(\|[^|]*\|)?\s*", entry.entry_str)
    if m:
        trainable_entry = entry.entry_str[m.span()[1]:].strip()
        if not trainable_entry:
            raise RuntimeError(f"Bad entry for {entry.title}: '{entry.entry_str}'")
    else:
        raise RuntimeError(f"Couldn't match {entry.title} on '{entry.entry_str}'")

In [ ]:
dictionary_path = "data/com_apple_MobileAsset_DictionaryServices_dictionaryOSX/69b7ab1cf0f75ad16bf6662b0a77fbfd36b7941f.asset/AssetData/New Oxford American Dictionary.dictionary/Contents/Resources/Body.data"
with open(dictionary_path, "rb") as f:
    valid_words = {e.title.upper() for e in dictionary_definition.DictionaryDefinition.gen_from_apple_dictionary(f)}

In [ ]:
model =  modeling.GPT2LMHeadWithWeightedLossModel.from_pretrained("models/dictionary-scale-10-lr5e-5").to("cuda")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [ ]:
words = dictionary_definition.generate_words(
    tokenizer, model, allow_proper_nouns=False, blacklist=valid_words, num=1000, max_iterations=40
)
words.sort(key=lambda x: x.title)
for w in words:
    print(f"{w} {w.entry_str}")

In [ ]:
with open("words.tsv", "w") as f:
    for word in words:
        f.write(f"{word.title}\t{word.entry_str}\n")